# IF3270 Pembelajaran Mesin - Tugas Besar 1D
## Anggota kelompok :
1. T. Antra Oksidian Tafly / 13517020
2. Christopher Billy S. / 13517050
3. Abel Stanley / 13517068
4. Ferdy Santoso / 13517116

In [0]:
# Imports
import pandas as pd
import MyC45
import ExternalModel as em

## I. Skema split train 90% dan test 10%, dan menampilkan kinerja serta confusion matrixnya

In [16]:
from MultiLayerPerceptron import MultiLayerPerceptron
from MyC45 import MyTree
from MyC45 import handleContinuousAttribute
from sklearn.utils import shuffle
import pickle
import numpy as np
import pandas as pd
import sys

def MLP(data, batch_size):

    #data = load_iris()
    # list1 = list(data.iloc[0,:-2])
    # print(list1)
    # target = list(data.iloc[0,4:])
    # print(target)
    mlp = MultiLayerPerceptron(_layers=1 , _bias=1, _inputs=4, _outputs=2, _learningRate=0.01, _maxIter=50)

    for i in range(mlp.maxIter):
        mlp.totalError = 0
        data = shuffle(data)
        #print(data.data[0])

        for j in range(len(data)):
            row = list(data.iloc[j,:-mlp.outputs])
            target = list(data.iloc[j,mlp.inputs:])
            mlp.estimate(row)
            #mlp.estimate(data.data[j])
            mlp.updateDeltaWeight(target)
            if j % batch_size == 0:
                mlp.updateWeight()
                mlp.updateTotalError(target)
                # print("at iteration ", i, " error = " + str(mlp.totalError))
        if j % batch_size != 0:
            mlp.updateWeight()
            mlp.updateTotalError(target)
            # print("at iteration ", i, " error = " + str(mlp.totalError))


        if(mlp.totalError < 0.01):
            break

    return mlp

dataMLP = pd.read_csv("irisMLP.csv")
training_data = dataMLP.sample(frac = 0.9)
test_dataMLP = dataMLP.drop(training_data.index)
mlp = MLP(training_data, 10)
confusion_matrixMLP = np.zeros((3, 3))
for i in range(len(test_dataMLP)):
    test = list(test_dataMLP.iloc[i,:-mlp.outputs])
    target = list(test_dataMLP.iloc[i,mlp.inputs:])
    estimate = mlp.estimate(test)
    # print(estimate)
    for i in range(len(estimate)):
        estimate[i] = round(estimate[i])
    # print(estimate)
    # print(target)
    row = -1
    if (target == [0, 0]):
        row = 0
    elif (target == [0, 1]):
        row = 1
    elif (target == [1, 1]):
        row = 2
    column = -1
    if (estimate == [0, 0]):
        column = 0
    elif (estimate == [0, 1]):
        column = 1
    elif (estimate == [1, 1]):
        column = 2
    if (column >= 0 and row >= 0):
        confusion_matrixMLP[row][column] += 1
print("MLP confusion matrix:")
print(confusion_matrixMLP)

# Penghitungan kinerja
total = 0
for i in range(len(confusion_matrixMLP)):
    for j in range(len(confusion_matrixMLP[i])):
        total+=confusion_matrixMLP[i][j]
if (total!=0):
    for i in range(len(confusion_matrixMLP)):
        print(i)
        p = 0
        p_acc = 0
        error_rate = 0
        tp = confusion_matrixMLP[i][i]
        print("tp=" + str(tp))
        fn = 0
        fp = 0
        for j in range(len(confusion_matrixMLP[i])):
            p += confusion_matrixMLP[i][j]
            p_acc += confusion_matrixMLP[j][i]
            if (j!=i):
                fn += confusion_matrixMLP[i][j]
                fp += confusion_matrixMLP[j][i]
        tn = total - (tp + fn + fp)
        n = total-p
        n_acc = total-p_acc
        print("p=" + str(p))
        print("p_acc=" + str(p_acc))
        print("fn=" + str(fn))
        print("fp=" + str(fp))
        print("tn=" + str(tn))
        print("n=" + str(n))
        print("n_acc=" + str(n_acc))

        print('error rate ' + str(i) + ":")
        error_rate = float(fn+fp) / total
        print(error_rate)

        if (p!=0):
            recall = float(tp)/p
            print('tp-rate/recall/sensitivity ' + str(i) + ":")
            print(recall)

        if (n!=0):
            fp_rate = float(fp)/n
            print('fp-rate ' + str(i) + ":")
            print(fp_rate)

        if (n!=0):
            specificity = float(tn)/n
            print('specificity ' + str(i) + ":")
            print(specificity)

        if (p_acc!=0):
            precision = float(tp)/p_acc
            print('precision ' + str(i) + ":")
            print(precision)

        if (n!=0):
            accuracy = float(tp + tn)/total
            print('accuracy ' + str(i) + ":")
            print(accuracy)

        if (precision+recall != 0):
            f_measure = 2 * float(precision * recall) / (precision + recall)
            print('F-measure ' + str(i) + ":")
            print(f_measure)

        balanced_accuracy = float(recall + specificity) / 2
        print('Balanced accuracy ' + str(i) + ":")
        print(balanced_accuracy)
    

    


dataDTL = pd.read_csv("irisDTL.csv")
dataDTL = handleContinuousAttribute(dataDTL)
training_dataDTL = dataDTL.sample(frac = 0.9)
test_dataDTL = dataDTL.drop(training_dataDTL.index)
tree = MyTree(_targetAttribute = "target")
tree.buildTreeInit(trainingSet=training_dataDTL)
confusion_matrixDTL = np.zeros((3, 3))
# tree.printTree()
for i in range(len(test_dataDTL)):
    test = test_dataDTL.iloc[i]
    target = test["target"]
    test = test.to_dict()
    for key in test:
        test[key] = str(test[key])
    # print(test)
    estimate = tree.predict(test)
    # print(estimate)
    row = -1
    row = int(target)
    column = -1
    column = int(estimate)
    if (column >= 0 and row >= 0):
        confusion_matrixDTL[row][column] += 1
print("DTL confusion matrix:")
print(confusion_matrixDTL)

# Penghitungan kinerja
total = 0
for i in range(len(confusion_matrixDTL)):
    for j in range(len(confusion_matrixDTL[i])):
        total+=confusion_matrixDTL[i][j]
if (total!=0):
    for i in range(len(confusion_matrixDTL)):
        print(i)
        p = 0
        p_acc = 0
        error_rate = 0
        tp = confusion_matrixDTL[i][i]
        print("tp=" + str(tp))
        fn = 0
        fp = 0
        for j in range(len(confusion_matrixDTL[i])):
            p += confusion_matrixDTL[i][j]
            p_acc += confusion_matrixDTL[j][i]
            if (j!=i):
                fn += confusion_matrixDTL[i][j]
                fp += confusion_matrixDTL[j][i]
        tn = total - (tp + fn + fp)
        n = total-p
        n_acc = total-p_acc
        print("p=" + str(p))
        print("p_acc=" + str(p_acc))
        print("fn=" + str(fn))
        print("fp=" + str(fp))
        print("tn=" + str(tn))
        print("n=" + str(n))
        print("n_acc=" + str(n_acc))

        print('error rate ' + str(i) + ":")
        error_rate = float(fn+fp) / total
        print(error_rate)

        if (p!=0):
            recall = float(tp)/p
            print('tp-rate/recall/sensitivity ' + str(i) + ":")
            print(recall)

        if (n!=0):
            fp_rate = float(fp)/n
            print('fp-rate ' + str(i) + ":")
            print(fp_rate)

        if (n!=0):
            specificity = float(tn)/n
            print('specificity ' + str(i) + ":")
            print(specificity)

        if (p_acc!=0):
            precision = float(tp)/p_acc
            print('precision ' + str(i) + ":")
            print(precision)

        if (n!=0):
            accuracy = float(tp + tn)/total
            print('accuracy ' + str(i) + ":")
            print(accuracy)

        if (precision+recall != 0):
            f_measure = 2 * float(precision * recall) / (precision + recall)
            print('F-measure ' + str(i) + ":")
            print(f_measure)

        balanced_accuracy = float(recall + specificity) / 2
        print('Balanced accuracy ' + str(i) + ":")
        print(balanced_accuracy)

MLP confusion matrix:
[[2. 0. 0.]
 [0. 8. 0.]
 [0. 5. 0.]]
0
tp=2.0
p=2.0
p_acc=2.0
fn=0.0
fp=0.0
tn=13.0
n=13.0
n_acc=13.0
error rate 0:
0.0
tp-rate/recall/sensitivity 0:
1.0
fp-rate 0:
0.0
specificity 0:
1.0
precision 0:
1.0
accuracy 0:
1.0
F-measure 0:
1.0
Balanced accuracy 0:
1.0
1
tp=8.0
p=8.0
p_acc=13.0
fn=0.0
fp=5.0
tn=2.0
n=7.0
n_acc=2.0
error rate 1:
0.3333333333333333
tp-rate/recall/sensitivity 1:
1.0
fp-rate 1:
0.7142857142857143
specificity 1:
0.2857142857142857
precision 1:
0.6153846153846154
accuracy 1:
0.6666666666666666
F-measure 1:
0.761904761904762
Balanced accuracy 1:
0.6428571428571428
2
tp=0.0
p=5.0
p_acc=0.0
fn=5.0
fp=0.0
tn=10.0
n=10.0
n_acc=15.0
error rate 2:
0.3333333333333333
tp-rate/recall/sensitivity 2:
0.0
fp-rate 2:
0.0
specificity 2:
1.0
accuracy 2:
0.6666666666666666
F-measure 2:
0.0
Balanced accuracy 2:
0.5
Entropy not 0 but already ran out attributes
Entropy not 0 but already ran out attributes
Entropy not 0 but already ran out attributes
DTL confusion

## II. Skema 10-fold cross validation, dan menampilkan kinerjanya

In [15]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
import csv
import pandas as pd

from MyC45 import MyTree, handleContinuousAttribute, resetThresholdDict, printThresholdDict
from MultiLayerPerceptron import MultiLayerPerceptron
#import MultiLayerPerceptron

def kFold(nFold, # how many folds
         dataset, # dataset to be kFold-ed
         ):
    split = len(dataset) // nFold
    result = []
    start_idx = 0
    for i in range(nFold):
        indices = list(range(0,start_idx)) + list(range(start_idx+split,len(dataset)))
        train = dataset[indices]
        test = dataset[start_idx:start_idx+split]
        result.append( (train,test) ) # tuple of ( train dataset, test dataset)
        start_idx += split

    return result


irisDataMLP = []
with open('irisMLP.csv') as f:
    freader = csv.reader(f, delimiter=',')
    for row in freader:
        irisDataMLP.append(row)
irisDataMLP = np.array(irisDataMLP)

# print(irisDataMLP)

# kf = KFold(n_splits=10, shuffle=False, random_state=1)
attrs=['attr1', 'attr2', 'attr3', 'attr4', 'target']

nFold = 10
MLPresult = kFold(nFold, irisDataMLP)

# Evaluate for MLP:
print("Evaluating MLP Model with k-fold: ")

# totalScore = 0
confusion_matrixMLP = np.zeros((3, 3))
for train, test in MLPresult:
    print("TRAIN: --------")
    with open ('dummy.csv', mode='w', newline="") as f:
        fwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in train: # train dataset
            fwriter.writerow(row)
    # ANN:
    data = pd.read_csv("dummy.csv")
    mlp2 = MultiLayerPerceptron(_layers=1 , _bias=1, _inputs=4, _outputs=2, _learningRate=0.01, _maxIter=50)
    # MLP(32, data)

    print("VALIDATION: --------")
    correct = 0
    for row in test: # test dataset
        values = []
        for val, attr in zip(row[:-1], attrs[:-1]):
            values.append(float(val))
        # print(values)
        estimate = mlp2.estimate(values)
        estimate = [int(estimate[0]), int(estimate[1])]
        target = [int(row[-2]), int(row[-1])]
        print(estimate, " vs ", target)
        
        row = -1
        if (target == [0, 0]):
            row = 0
        elif (target == [0, 1]):
            row = 1
        elif (target == [1, 1]):
            row = 2
        column = -1
        if (estimate == [0, 0]):
            column = 0
        elif (estimate == [0, 1]):
            column = 1
        elif (estimate == [1, 1]):
            column = 2
        if (column >= 0 and row >= 0):
            confusion_matrixMLP[row][column] += 1
        # threshold: 0.5
#         if abs(prediction - float(row[-1])) < 0.5:
#             confusionMatrix[]
#             correct+=1

    print("MLP confusion matrix:")
    print(confusion_matrixMLP)

    # Penghitungan kinerja
    total = 0
    for i in range(len(confusion_matrixMLP)):
        for j in range(len(confusion_matrixMLP[i])):
            total+=confusion_matrixMLP[i][j]
    if (total!=0):
        for i in range(len(confusion_matrixMLP)):
            print(i)
            p = 0
            p_acc = 0
            error_rate = 0
            tp = confusion_matrixMLP[i][i]
            print("tp=" + str(tp))
            fn = 0
            fp = 0
            for j in range(len(confusion_matrixMLP[i])):
                p += confusion_matrixMLP[i][j]
                p_acc += confusion_matrixMLP[j][i]
                if (j!=i):
                    fn += confusion_matrixMLP[i][j]
                    fp += confusion_matrixMLP[j][i]
            tn = total - (tp + fn + fp)
            n = total-p
            n_acc = total-p_acc
            print("p=" + str(p))
            print("p_acc=" + str(p_acc))
            print("fn=" + str(fn))
            print("fp=" + str(fp))
            print("tn=" + str(tn))
            print("n=" + str(n))
            print("n_acc=" + str(n_acc))

            print('error rate ' + str(i) + ":")
            error_rate = float(fn+fp) / total
            print(error_rate)

            if (p!=0):
                recall = float(tp)/p
                print('tp-rate/recall/sensitivity ' + str(i) + ":")
                print(recall)

            if (n!=0):
                fp_rate = float(fp)/n
                print('fp-rate ' + str(i) + ":")
                print(fp_rate)

            if (n!=0):
                specificity = float(tn)/n
                print('specificity ' + str(i) + ":")
                print(specificity)

            if (p_acc!=0):
                precision = float(tp)/p_acc
                print('precision ' + str(i) + ":")
                print(precision)

            if (n!=0):
                accuracy = float(tp + tn)/total
                print('accuracy ' + str(i) + ":")
                print(accuracy)

            if (precision+recall != 0):
                f_measure = 2 * float(precision * recall) / (precision + recall)
                print('F-measure ' + str(i) + ":")
                print(f_measure)

            balanced_accuracy = float(recall + specificity) / 2
            print('Balanced accuracy ' + str(i) + ":")
            print(balanced_accuracy)

#     print("correct:", correct, "out of ", len(test))
#     totalScore += correct / float(len(test))

    
        
# print("MLP Model total score:", totalScore/nFold)

# Evaluate for C45:
irisData = []
with open('iris.csv') as f:
    freader = csv.reader(f, delimiter=',')
    for row in freader:
        irisData.append(row)
irisData = np.array(irisData)

C45result = kFold(nFold, irisData)
# print(C45result)
print("Evaluating C45 Model with k-fold: ")

totalScore = 0
# idx = 0
for train, test in C45result:
    # print(idx)
    # idx+=1
    print("TRAIN: --------")
    # trainn, testt = C45result[7]
    with open ('dummy.csv', mode='w') as f:
        fwriter = csv.writer(f)
        for row in train:
            fwriter.writerow(row)

    data = pd.read_csv("dummy.csv", header=None, names=attrs)
    training_dataset = handleContinuousAttribute(data)
    t2 = MyTree(_targetAttribute = 'target')
    print("=================================================")
    t2.buildTreeInit(trainingSet = training_dataset)
    t2.printTree()
    print("=================================================")

    print("VALIDATION: --------")
    #print(irisData[test])
    correct = 0
    for row in test:
        values = {}
        for val, attr in zip(row[:-1], attrs[:-1]):
            values[attr] = float(val)
        # print(values)
        prediction = t.predict(values)
        print(prediction, " vs ", row[-1])
        if prediction == row[-1]:
            correct+=1
    print("correct:", correct, "out of ", len(test))
    totalScore += correct / float(len(test))
    # reset threshold dict:
    resetThresholdDict()

print("C45 Model total score:", totalScore/nFold)

Evaluating MLP Model with k-fold: 
TRAIN: --------
VALIDATION: --------
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
[0, 0]  vs  [0, 0]
MLP confusion matrix:
[[15.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
0
tp=15.0
p=15.0
p_acc=15.0
fn=0.0
fp=0.0
tn=0.0
n=0.0
n_acc=0.0
error rate 0:
0.0
tp-rate/recall/sensitivity 0:
1.0
precision 0:
1.0
F-measure 0:
1.0
Balanced accuracy 0:
0.5
1
tp=0.0
p=0.0
p_acc=0.0
fn=0.0
fp=0.0
tn=15.0
n=15.0
n_acc=15.0
error rate 1:
0.0
fp-rate 1:
0.0
specificity 1:
1.0
accuracy 1:
1.0
F-measure 1:
1.0
Balanced accuracy 1:
1.0
2
tp=0.0
p=0.0
p_acc=0.0
fn=0.0
fp=0.0
tn=15.0
n=15.0
n_acc=15.0
error rate 2:
0.0
fp-rate 2:
0.0
specificity 2:
1.0
accuracy 2:
1.0
F-measure 2:
1.0
Balanced accuracy 2:
1.0
TRAIN: --------
VALIDATION: --------
[0, 0]  vs  

## III. Menyimpan (save) model pembelajaran

In [0]:
# Read data
data = pd.read_csv("iris.csv", header=None, names=['attr1', 'attr2', 'attr3', 'attr4', 'target'])
data = MyC45.handleContinuousAttribute(data)

# Learning
# t = MyC45.MyTree(_targetAttribute = 'target')
# t.buildTreeInit(trainingSet = data)

# Save model
em.saveModel(tree, "split_tree.pickle")
em.saveModel(mlp, "split_mlp.pickle")
em.saveModel(t, "myc45_model.pickle")
em.saveModel(mlp2, "mlp_kfold.pickle")
em.saveModel(t2,"myc45_model_kfold.pickle")

Entropy not 0 but already ran out attributes
Entropy not 0 but already ran out attributes
Entropy not 0 but already ran out attributes
Entropy not 0 but already ran out attributes
Model saved
Model saved
Model saved
Model saved
Model saved


## IV. Membaca (load) model pembelajaran

# Load model
t_new = em.loadModel("myc45_model.pickle")
split_tree_new = em.loadModel("split_tree.pickle")
split_mlp_new = em.loadModel("split_mlp.pickle")
mlp2_kfold_new = em.loadModel("mlp_kfold.pickle")
t2_kfold_new = em.loadModel("myc45_model_kfold.pickle")

confusion_matrixMLP = np.zeros((3, 3))
for i in range(len(test_dataMLP)):
    test = list(test_dataMLP.iloc[i,:-mlp.outputs])
    target = list(test_dataMLP.iloc[i,mlp.inputs:])
    estimate = split_mlp_new.estimate(test)
    # print(estimate)
    for i in range(len(estimate)):
        estimate[i] = round(estimate[i])
    # print(estimate)
    # print(target)
    row = -1
    if (target == [0, 0]):
        row = 0
    elif (target == [0, 1]):
        row = 1
    elif (target == [1, 1]):
        row = 2
    column = -1
    if (estimate == [0, 0]):
        column = 0
    elif (estimate == [0, 1]):
        column = 1
    elif (estimate == [1, 1]):
        column = 2
    if (column >= 0 and row >= 0):
        confusion_matrixMLP[row][column] += 1
print("MLP confusion matrix:")
print(confusion_matrixMLP)

confusion_matrixDTL = np.zeros((3, 3))
# tree.printTree()
for i in range(len(test_dataDTL)):
    test = test_dataDTL.iloc[i]
    target = test["target"]
    test = test.to_dict()
    for key in test:
        test[key] = str(test[key])
    # print(test)
    estimate = split_tree_new.predict(test)
    # print(estimate)
    row = -1
    row = int(target)
    column = -1
    column = int(estimate)
    if (column >= 0 and row >= 0):
        confusion_matrixDTL[row][column] += 1
print("DTL confusion matrix:")
print(confusion_matrixDTL)


## V. Klasifikasi menggunakan model pada poin IV

# print(t_new.predict({}))
# ANN
values = [1.2, 3.2, 9.9, 4.3]
prediction = mlp2_kfold_new.estimate(values)
# DTL
t2_kfold_new = MyC45.MyTree(_targetAttribute = 'target')
t2_kfold_new.buildTreeInit(trainingSet = data)
values = {
   'attr1' : 1.2,
   'attr2' : 2.3,
   'attr3' : 9.9,
   'attr4' : 3.4
}
prediction = t.predict(values)

## VI. Analisis hasil eksperimen poin I dan II
    
### Split train 90% dan test 10%
    Pada eksperimen point I Skema split train 90% dan test 10% bagian MLP, menghasilkan confusion matrix<br>

[5 0 0<br>
0 6 0<br>
0 3 1]<br>
   
    dimana pada row 0 dan row 1, model kami berhasil memprediksi seluruh kasus dengan benar sedangkan pada row 2, model kami hanya memprediksi 1 dari 4 saja yang benar. hal ini menunjukkan bahwa ada kasus dimana model kami mengembalikan false positive.
    
    Lalu pada training DTL dengan Skema I model kami menghasilkan confusion matrix sebagai berikut <br>
[6 0 0<br>
1 3 1<br>
0 0 4]<br>

    Pada row 0 model kami berhasil memprediksi true positive seluruhnya. lalu pada row 1, model kami berhasil memprediksi benar 3 dari 5 namun untuk sisa 2nya salah. Selain itu pada row 2, model kami berhasil memprediksi true positive seluruh kasus. Salah satu faktor penyebab kesalahan dalam memprediksi data adalah faktor shuffle dalam training, kurangnya iterasi, overfit data, kurang spesifiknya data yang tersedia, atau bisa juga kurang banyaknya data. Untuk MLP, beberapa faktor lainnya adalah karena kurangnya layer pada hidden node sehingga mengurangi akurasi dan juga faktor batch size yang tidak seimbang.

### K-Fold Cross Validation
    K-Fold Cross-Validation adalah resampling procedure yang digunakan untuk evaluasi model ML ketika dataset yang tersedia sangat sedikit. Prosedur ini memiliki parameter tunggal k yang menandakan seberapa banyak grup sample data yang diberikan akan terbagi. 

    Untuk kasus ini, k = 10 (ten fold cross validation). 10 dipilih karena menurut penelitian telah dibuktikan k yang bernilai 10 memiliki probabilitas untuk menghasilkan evaluasi model ML yang memiliki bias rendah dan variansi yang minimal.

#### Prosedur yang dilakukan untuk k-Fold:
1.  Sebisa mungkin split dataset original menjadi group sebanyak k
2. Untuk setiap group k:
     a. Ambil satu partisi sebagai validation dataset
     b. dataset yang tersisa dijadikan training dataset
     c. gunakan dataset training untuk membangun dan melatih model ML
     d. menghitung skor evaluasi dengan validation dataset dan buang model tersebut
3. Tarik kesimpulan /rata-rata skor dari model ML menurut setiap skor dari tahap evaluasi

#### Constraint:
1. Setiap group harus unique, setiap observasi (row) pada dataset hanya boleh menjadi bagian dari validation set hanya 1x saja
2. Suatu observasi dapat menjadi bagian dari training set atau validation set pada suatu iterasi, namun tidak kedua-duanya sekaligus dalam satu iterasi

|Iteration|Training set obsevations|Testing set observations|
|---------|------------------------|------------------------|
|1|[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]|[0 1 2 3 4]|
|2|[ 0  1  2  3  4 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]|[5 6 7 8 9]|
|3|[ 0  1  2  3  4  5  6  7  8  9 15 16 17 18 19 20 21 22 23 24]|[10 11 12 13 14]|
|4|[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 20 21 22 23 24]|[15 16 17 18 19]|
|5|[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]|[20 21 22 23 24]|
    
    
### Kesimpulan
    Jika dibandingkan dengan 10 fold Cross Validation, tentunya ada kelemahan dan kelebihan dari skema ke-2 ini. Dimana untuk Skema 10 Fold Cross Validation, training dilakukan 10 kali dengan men-switch training data dan test data yang ada. Dari sini, dapat kita temukan pasangan training data dan test data yang memiliki accuracy paling besar dari 10 kali iterasi pelatihan model. 

    Kelemahan dari 10 fold Cross Validation tentunya terletak pada waktu yang diberikan untuk melatih modelnya. Dimana jika kita menggunakan 10 Fold Cross Validation, tentunya pelatihan model akan lebih lama namun akan mendapatkan model yang lebih akurat. Kenapa 10? karena nilai 10 adalah nilai yang telah ditemukan melalui eksperimen untuk secara umum menghasilkan estimasi keterampilan model dengan bias rendah dan varian yang tidak terlalu bervariasi.